# Load Data Step:
* AWS MySQL Backup, S3 Upload, Redshift Cluster Setup, Table Creation, upload data using S3 cp command and Cleanup


* Mounts Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* Installing Required Python Packages: boto3 and paramiko


In [2]:
!pip install boto3 paramiko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 50.7 MB/s eta 0:00:00


* AWS and Paramiko Library Importation

In [19]:
import boto3
import paramiko
import psycopg2
import time
import subprocess

* AWS Credential and Parameter Configuration for Redshift and S3
* you can change parameter value according to your need

In [41]:
#AWS Credential with Region
aws_access_key = 'your aws_access_key'
aws_secret_key = 'your aws_secret_key'
aws_region = 'ap-south-1'

# IAM role ARN
# go to IAM Role and search this iam_role_arn role value and then you will find actual role copy arn of that role and past here
iam_role_arn = 'AmazonRedshift-CommandsAccessRole-20231213T202016' # must be replace this role

# S3 bucket parameter configuration:
s3_bucket_name = 'etlpro'
s3_file_path = 'Backup_imdb.csv'

# Specify Redshift parameters
cluster_identifier = 'clustername'
database_name = 'imdb'
master_username = 'admin'
master_password = 'l4Fwt@!~`=gg<Uff&^%$hyru' # Note : password must be strong either it will be show an error 
node_type = 'dc2.large'
number_of_nodes = 2

* SSH into AWS EC2 Instance and Execute Commands for MySQL Backup and S3 Upload


* Note : if File uploading error then attach S3 permission role to ec2, still not resolve than do aws configure in ec2

In [23]:
# SSH into the instance (you might need to replace the key file path and username)
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# if you same .pem file in other location then replace the path
key_file_path = '/content/drive/MyDrive/Colab Notebooks/ETL_KP.pem'
public_ip_address = 'Your_Ec2_instance_public_ip_address'

# while you use different type of operating system then must be change the username
ssh_client.connect(public_ip_address, username='ubuntu', key_filename=key_file_path)

# commands for Backup a database, Download .sql file, and upload to S3
# must be change sql password in commands at your_sql_user_password
commands = [
    'sudo mysqldump -u etl -pyour_sql_user_password imdb > /tmp/Backup_imdb.sql',
    'sudo apt-get update',
    'sudo apt-get upgrade -y',
    'sudo apt-get install awscli -y',
    'aws s3 cp /tmp/Backup_imdb.sql s3://etlpro/'
]

# Execute each command
for command in commands:
    stdin, stdout, stderr = ssh_client.exec_command(command)
    output = stdout.read().decode('utf-8')
    error = stderr.read().decode('utf-8')

    # Print the output and error messages
    print(f"Command Output for '{command}':\n{output}")
    print(f"Command Error for '{command}':\n{error}")

# Close the SSH connection
ssh_client.close()


Command Output for 'sudo mysqldump -u etl -pAdmin@123 imdb > /tmp/Backup_imdb.sql':

Command Error for 'sudo mysqldump -u etl -pAdmin@123 imdb > /tmp/Backup_imdb.sql':
mysqldump: [Warning] Using a password on the command line interface can be insecure.
mysqldump: Error: 'Access denied; you need (at least one of) the PROCESS privilege(s) for this operation' when trying to dump tablespaces

Command Output for 'sudo apt-get update':
Hit:1 http://ap-south-1.ec2.archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://ap-south-1.ec2.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://ap-south-1.ec2.archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists...

Command Error for 'sudo apt-get update':

Command Output for 'sudo apt-get upgrade -y':
Reading package lists...
Building dependency tree...
Reading state information...
Calculating upgrade...
The following packages have been kept back:
  linux-aw

* AWS Redshift Client Initialization


In [54]:
# Create object for Redshift cluster
redshift_client = boto3.client('redshift', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=aws_region)

* Creating AWS Redshift Cluster:


In [55]:
response = redshift_client.create_cluster(
    ClusterIdentifier=cluster_identifier,
    NodeType=node_type,
    MasterUsername=master_username,
    MasterUserPassword=master_password,
    NumberOfNodes=number_of_nodes,
    DBName=database_name,
    IamRoles=[iam_role_arn,]
)

print(f"Cluster {cluster_identifier} is being created. Please wait...")

# Wait for the cluster to be available
redshift_client.get_waiter('cluster_available').wait(ClusterIdentifier=cluster_identifier)

print(f"Cluster {cluster_identifier} has been created successfully.")


Cluster etlproject is being created. Please wait...
Cluster etlproject has been created successfully.


* Describing AWS Redshift Cluster:


In [56]:
# Describe the cluster to obtain its details
describe_cluster = redshift_client.describe_clusters(ClusterIdentifier=cluster_identifier)

# Connect to the cluster and run example queries
cluster_endpoint = describe_cluster['Clusters'][0]['Endpoint']['Address']
port = describe_cluster['Clusters'][0]['Endpoint']['Port']
cluster_status = describe_cluster['Clusters'][0]['ClusterStatus']
cluster = describe_cluster['Clusters'][0]
cluster_role = describe_cluster['Clusters'][0]['IamRoles']

print(cluster_endpoint)
print(port)
print(cluster_status)
print(cluster_role)
print(cluster)



etlproject.ciajlcnv4lg2.ap-south-1.redshift.amazonaws.com
5439
available
[{'IamRoleArn': 'arn:aws:iam::475109786741:role/service-role/AmazonRedshift-CommandsAccessRole-20231213T202016', 'ApplyStatus': 'in-sync'}]
{'ClusterIdentifier': 'etlproject', 'NodeType': 'dc2.large', 'ClusterStatus': 'available', 'ClusterAvailabilityStatus': 'Unavailable', 'MasterUsername': 'admin', 'DBName': 'imdb', 'Endpoint': {'Address': 'etlproject.ciajlcnv4lg2.ap-south-1.redshift.amazonaws.com', 'Port': 5439}, 'ClusterCreateTime': datetime.datetime(2023, 12, 15, 19, 7, 37, 116000, tzinfo=tzlocal()), 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0dae590430548e22c', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'default', 'VpcId': 'vpc-051c6bf2ff976c0e8', 'AvailabilityZone': 'ap-south-1c', 'Pr

* Creating Connection to AWS Redshift Cluster


In [57]:
# Create a connection string
conn_string = f"dbname={database_name} user={master_username} password={master_password} host={cluster_endpoint} port={port}"

conn = psycopg2.connect(conn_string)

print("Connection established successfully!")

cursor = conn.cursor()

Connection established successfully!


* Redshift Table Creation and Data Loading


In [ ]:
# CREATE TABLE command to the Redshift cluster
# if your csv data are different then please change the table schema
create_table_command = """
CREATE TABLE IF NOT EXISTS imdb.public.movie_data (
    movie_name VARCHAR(255) NOT NULL UNIQUE,
    year_of_release INTEGER NOT NULL,
    watch_time VARCHAR(30) NOT NULL,
    rating REAL NOT NULL,
    metascore INTEGER NOT NULL,
    votes INTEGER NOT NULL,
    avg_rating REAL NOT NULL,
    id INTEGER PRIMARY KEY NOT NULL
);
"""

# Execute the CREATE TABLE command
cursor.execute(create_table_command)

# Generate the COPY command
copy_command = f"""
COPY imdb.public.movie_data
FROM 's3://{s3_bucket_name}/{s3_file_path}'
IAM_ROLE '{iam_role_arn}'
FORMAT AS CSV
DELIMITER ','
QUOTE '"'
IGNOREHEADER 1
REGION AS 'ap-south-1';
"""

# Execute the COPY command to load data and automatically create the table
cursor.execute(copy_command)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

* Deleting AWS Redshift Cluster:

In [59]:
cluster_identifier_to_delete = cluster_identifier

# Delete the Redshift cluster
response = redshift_client.delete_cluster(ClusterIdentifier=cluster_identifier_to_delete, SkipFinalClusterSnapshot=True)
print(f"Cluster {cluster_identifier_to_delete} is being deleted. Please wait...")

# Wait for the cluster to be deleted
redshift_client.get_waiter('cluster_deleted').wait(ClusterIdentifier=cluster_identifier_to_delete)

print(f"Cluster {cluster_identifier_to_delete} has been deleted successfully.")


Cluster etlproject is being deleted. Please wait...
Cluster etlproject has been deleted successfully.
